## setup

In [1]:
import Interface as I

/scratch/visualn/bzfharth/in-silico-install-dir/project_src/in_silico_framework/model_data_base/distributed_lock.py:13: UserWarning: environment variable ISF_DISTRIBUTED_LOCK_CONFIG is not set. Falling back to default configuration.
  warnings.warn('environment variable ISF_DISTRIBUTED_LOCK_CONFIG is not set. ' +
/scratch/visualn/bzfharth/in-silico-install-dir/project_src/in_silico_framework/model_data_base/distributed_lock.py:33: UserWarning: Using file based locking.Please be careful on nfs mounts as file based locking has issues in this case.
  warnings.warn('Using file based locking.'


trying to connect to distributed locking server {'type': 'redis', 'config': {'host': 'spock', 'port': 8885, 'socket_timeout': 1}}
trying to connect to distributed locking server {'type': 'redis', 'config': {'host': 'localhost', 'port': 6379, 'socket_timeout': 1}}
trying to connect to distributed locking server {'type': 'file'}
Current version: heads/vcbm-2023+0.gcd990fa3.dirty
Current pid: 1067361


/scratch/visualn/bzfharth/in-silico-install-dir/anaconda_isf3/lib/python3.8/site-packages/pandas_msgpack-0.1.4+14.gfcb0471-py3.8-linux-x86_64.egg/pandas_msgpack/packers.py:78: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import (Timestamp, Period, Series, DataFrame,  # noqa
/scratch/visualn/bzfharth/in-silico-install-dir/project_src/in_silico_framework/visualize/cell_morphology_visualizer.py:29: ImportWarning: Dash could not be imported. Interactive visualisations will not work.
  warnings.warn(


Loading mechanisms:
setting up local multiprocessing framework ... done


Loaded modules with __version__ attribute are:
IPython: 7.19.0, Interface: heads/vcbm-2023+0.gcd990fa3.dirty, PIL: 8.2.0, _csv: 1.0, _ctypes: 1.1.0, _curses: b'2.2', _decimal: 1.70, argparse: 1.1, backcall: 0.2.0, blosc: 1.10.2, bluepyopt: 1.9.126, bottleneck: 1.3.2, cffi: 1.14.3, click: 7.1.2, cloudpickle: 1.6.0, colorama: 0.4.4, csv: 1.0, ctypes: 1.1.0, cycler: 0.10.0, cytoolz: 0.11.0, dask: 2.30.0, dateutil: 2.8.1, deap: 1.3, decimal: 1.70, decorator: 4.4.2, distributed: 2.30.1, distutils: 3.8.5, fsspec: 0.8.3, future: 0.18.2, ipykernel: 5.3.4, ipython_genutils: 0.2.0, ipywidgets: 7.5.1, jedi: 0.17.1, jinja2: 2.11.2, joblib: 0.17.0, json: 2.0.9, jupyter_client: 6.1.7, jupyter_core: 4.6.3, kiwisolver: 1.3.0, logging: 0.5.1.2, markupsafe: 1.1.1, matplotlib: 3.3.2, mkl: 2.3.0, neuron: 7.8.2, numexpr: 2.7.1, numpy: 1.19.2, pandas: 1.1.3, pandas_msgpack: 0.1.4+14.gfcb0471.dirty, parameters: 0.2.1, parso: 0.7.0, pas

/scratch/visualn/bzfharth/in-silico-install-dir/project_src/in_silico_framework/Interface.py:190: UserWarning:

The source folder has uncommited changes!



In [2]:
postfix = "_2023-06-22"
mdb = I.ModelDataBase('/scratch/visualn/bzfharth/simulation_data'+postfix)
print(mdb.keys())

['89', '90', 'WR69', '88', '93', '84', '87', 'WR64', '85', '86', '91', 'WR71', 'examplary_models', 'df_all', 'run_gillis_protocol_v1', 'all_models_py3', 'run_gillis_protocol_v1__all_models_genetic_py3', 'all_models_genetic_py3', 'all_models_genetic_py2_1', 'all_models_genetic_py2_2', 'all_models_RW_BAC2step_seed_11_filtered', 'all_models_RW_BAC2step_seed_17_filtered', 'all_models_RW_BAC2step_seed_1_filtered']


In [3]:
import os
import shutil

def makeCleanDir(dirname):
    if(os.path.exists(dirname)):
        shutil.rmtree(dirname, ignore_errors=False, onerror=None)
    os.mkdir(dirname)

selected_dataset = 'all_models_RW_BAC2step_seed_1_filtered'
csv_outfolder = os.path.join('/scratch/visualn/bzfharth/simulation_data_postprocessed', selected_dataset+postfix)

makeCleanDir(csv_outfolder)

In [4]:
# init
import pickle
import biophysics_fitting.hay_evaluation
import project_specific_ipynb_code.biophysical_models
from project_specific_ipynb_code.biophysical_models import get_model_ddf_from_encapsulating_mdb, objectives_BAC
from project_specific_ipynb_code.biophysical_models import get_ddf_selected
from project_specific_ipynb_code.hot_zone import get_cell_object_from_hoc
from project_specific_ipynb_code.biophysical_models import min_max_plot, normalize_params
from project_specific_ipynb_code.biophysical_models import CurrentAnalysis
from project_specific_ipynb_code.biophysical_models import modify_simulator_to_record_apical_dendrite_conductances

In [5]:
def clean_df(df, max_nans=3000):
    blocked_columns = []
    columns_many_nan = df.columns[df.isna().sum() > max_nans].to_list()
    boolean_columns = df.select_dtypes(include=bool).columns.to_list()
    object_columns = df.select_dtypes(include=object).columns.to_list()
    
    columns_to_drop = blocked_columns + columns_many_nan + boolean_columns + object_columns

    df_pruned_cols = df.drop(columns_to_drop, axis=1)
    df_cleaned = df_pruned_cols.dropna()
    return df_cleaned



In [6]:
folder = mdb[selected_dataset]
folder

'/scratch/visualn/bzfharth/simulation_data_2023-06-22/all_models_rw_bac2step_seed_1_filtered_rqv6y_nj_'

In [9]:
import pandas_msgpack
import glob

files = glob.glob(os.path.join(folder, "pandas_to_msgpack_*"))
files_dict = {}
for file in files:
    idx = int(file.split("_")[-1])
    files_dict[idx] = file

file_indices = sorted(files_dict.keys())
print(len(file_indices))

1000


In [10]:
max_idx = 10
for file_idx in file_indices[0:max_idx]:
    df = pandas_msgpack.read_msgpack(files_dict[file_idx])        
    df.to_csv(os.path.join(csv_outfolder, "simulations_uncleaned_{}.csv".format(file_idx)), index=False)
    df_cleaned = clean_df(df)    
    df_cleaned.to_csv(os.path.join(csv_outfolder, "simulations_{}.csv".format(file_idx)), index=False)
df_cleaned

,ephys.CaDynamics_E2_v2.apic.decay,ephys.CaDynamics_E2_v2.apic.gamma,ephys.CaDynamics_E2_v2.axon.decay,ephys.CaDynamics_E2_v2.axon.gamma,ephys.CaDynamics_E2_v2.soma.decay,ephys.CaDynamics_E2_v2.soma.gamma,ephys.Ca_HVA.apic.gCa_HVAbar,ephys.Ca_HVA.axon.gCa_HVAbar,ephys.Ca_HVA.soma.gCa_HVAbar,ephys.Ca_LVAst.apic.gCa_LVAstbar,...,bAP_att2.raw,bAP_att2.normalized,bAP_att2,bAP_att3.raw,bAP_att3.normalized,bAP_att3,bAP_spikecount.raw,bAP_spikecount.normalized,bAP_spikecount,bAP.err
0,101.176488,0.001968,57.969940,0.008433,512.051030,0.000931,0.000009,0.000305,0.000209,0.196167,...,31.795874,1.320413,1.320413,23.623914,1.326483,1.326483,1,0.0,0.0,249.647085
1,100.966144,0.001938,56.727459,0.008466,511.452345,0.000910,0.000015,0.000306,0.000208,0.196072,...,31.590493,1.340951,1.340951,23.436450,1.346576,1.346576,1,0.0,0.0,249.646382
2,100.712629,0.001908,59.754435,0.008560,514.300023,0.001054,0.000020,0.000306,0.000207,0.196597,...,32.719118,1.228088,1.228088,24.541620,1.228122,1.228122,1,0.0,0.0,249.645791
3,101.768611,0.001861,55.564619,0.008563,513.289490,0.001039,0.000032,0.000309,0.000211,0.197135,...,32.973104,1.202690,1.202690,24.786234,1.201904,1.201904,1,0.0,0.0,249.648335
4,101.073808,0.002035,56.894286,0.008466,513.322712,0.000973,0.000003,0.000304,0.000208,0.195940,...,31.936605,1.306340,1.306340,23.756041,1.312321,1.312321,1,0.0,0.0,249.646356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67376,50.343413,0.000782,118.300272,0.003013,483.460566,0.000715,0.000750,0.000657,0.000282,0.102997,...,31.596905,1.340310,1.340310,23.547375,1.334686,1.334686,1,0.0,0.0,249.620447
67377,50.148264,0.000982,115.605117,0.003182,480.146506,0.000591,0.000733,0.000656,0.000282,0.102722,...,29.797245,1.520275,1.520275,21.874325,1.514006,1.514006,1,0.0,0.0,249.618146
67378,49.459440,0.000847,113.448365,0.003069,481.247953,0.000727,0.000727,0.000654,0.000280,0.103396,...,29.163244,1.583676,1.583676,21.298567,1.575716,1.575716,1,0.0,0.0,249.616303
67379,49.717444,0.001064,117.128577,0.003048,481.906999,0.000507,0.000723,0.000658,0.000285,0.103037,...,31.353568,1.364643,1.364643,23.295975,1.361632,1.361632,1,0.0,0.0,249.616461
